# Face Recognition — Google Colab (gcolab) notebook

This notebook sets up a Colab-style environment, installs dependencies, mounts Google Drive, and runs the evaluation (`--eval`) of the `main.py` face recognition project. Select Runtime -> Change runtime type -> GPU before running cells for best performance.

Sections:
1. Install and import `gcolab` + dependencies
2. Authenticate & mount Google Drive
3. Initialize `gcolab` session and check runtime
4. Upload / download files (Kaggle support)
5. Sync workspace with Drive
6. Run evaluation and save metrics
7. Tests and troubleshooting tips


In [ ]:
# 1) Install dependencies
# Run these cells once. GPU runtime recommended (Runtime -> Change runtime type -> GPU).
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 || pip install torch torchvision
!pip install opencv-python-headless
!pip install retina-face
!pip install deepface
!pip install kaggle
!pip install gdown
# Optional: `gcolab` is a Colab helper and is not published on PyPI.
# Skip installing it when running locally — Colab provides it automatically.

# Verify packages
import sys, torch
print('Python', sys.version)
print('Torch CUDA available:', torch.cuda.is_available())


---

## 2) Authenticate & mount Google Drive

Use the cell below to mount your Google Drive. Place your project folder (or upload the repo zip) under `MyDrive/face_recognition`.

If you want to use Kaggle datasets, upload your `kaggle.json` to the notebook (or place it in `~/.kaggle/kaggle.json`).


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Set project path on Drive (adjust if you use a different folder)
DRIVE_PROJECT = '/content/drive/MyDrive/face_recognition'

import os
if os.path.exists(DRIVE_PROJECT):
    print('Found project folder on Drive:', DRIVE_PROJECT)
else:
    print('Drive mounted, create or upload your project to', DRIVE_PROJECT)


---

## 3) Initialize gcolab session & check runtime

This cell shows how to initialize `gcolab` (if you want to use its helpers) and prints runtime info (GPU/CPU, Python version). If you don't want to use `gcolab`, skip these cells — mounting Drive and running shell commands works fine without it.


In [ ]:
# Initialize gcolab (optional)
try:
    import gcolab
    print('gcolab version:', gcolab.__version__)
except Exception as e:
    print('gcolab not available or failed to import:', e)

# Runtime info
import platform
print('Python', platform.python_version())

import torch
if torch.cuda.is_available():
    !nvidia-smi
else:
    print('CUDA not available')


---

## 4) Upload / Download Files (Kaggle support)

If you need a dataset from Kaggle, upload your `kaggle.json` (Account -> Create API token) and run the snippet below to download datasets. Alternatively, upload a zipped project to Drive and unzip into the workspace.


In [ ]:
# Example: use Kaggle CLI to download a dataset (optional)
# Upload kaggle.json to /root/.kaggle/kaggle.json first (use file upload in Colab UI)
import os
if os.path.exists('/root/.kaggle/kaggle.json'):
    print('Found kaggle.json')
else:
    print('If you need Kaggle, upload kaggle.json to /root/.kaggle/kaggle.json via the file upload UI')

# Example command (uncomment and edit to use):
# !kaggle datasets download -d <owner/dataset-name> -p /content --unzip


---

## 5) Sync Local Workspace with Drive

Copy your project from Drive into the runtime workspace (`/content`) so the notebook can run `main.py` and use local paths.


In [ ]:
# Copy project from Drive into /content (adjust path to your Drive project folder)
import shutil
if os.path.exists(DRIVE_PROJECT):
    print('Copying project to runtime...')
    dst = '/content/face_recognition'
    if os.path.exists(dst):
        print('Removing existing runtime copy...')
        shutil.rmtree(dst)
    shutil.copytree(DRIVE_PROJECT, dst)
    %cd /content/face_recognition
    print('Files copied. Current dir:', os.getcwd())
else:
    print('Drive project not found; upload your project to the Drive path or upload files via the Colab UI')


---

## 6) Run evaluation and save metrics

This cell runs `main.py --eval --fast` and captures output to `metrics.txt` and `metrics.json` (basic parsing). Adjust the command if your `main.py` path differs.


In [ ]:
# Run evaluation and capture output
import subprocess, shlex, json, re
cmd = 'python main.py --eval --fast'
print('Running:', cmd)
proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
out_lines = []
for line in proc.stdout:
    print(line, end='')
    out_lines.append(line)
proc.wait()
with open('metrics.txt', 'w', encoding='utf-8') as f:
    f.writelines(out_lines)
print('\nMetrics written to metrics.txt')

# Simple parsing example (extract key metrics)
metrics = {}
for L in out_lines:
    m = re.match(r"Overall Accuracy: ([0-9\.]+) %", L)
    if m:
        metrics['overall_accuracy'] = float(m.group(1))
    m = re.match(r"AUC: ([0-9\.]+)", L)
    if m:
        metrics['auc'] = float(m.group(1))
    m = re.match(r"EER: ([0-9\.]+)", L)
    if m:
        metrics['eer'] = float(m.group(1))

with open('metrics.json','w') as f:
    json.dump(metrics, f, indent=2)
print('Parsed metrics saved to metrics.json', metrics)


---

## 7) Download results and save to Drive

Use the cell below to download `metrics.txt` or copy it back to Drive.


In [ ]:
# Download the metrics file to local machine (Colab will show a download link)
from google.colab import files
files.download('metrics.txt')

# Or copy to Drive
import shutil
if os.path.exists('/content/metrics.txt'):
    shutil.copy('/content/metrics.txt', os.path.join(DRIVE_PROJECT, 'metrics.txt'))
    print('Copied metrics.txt to Drive:', os.path.join(DRIVE_PROJECT, 'metrics.txt'))
else:
    print('metrics.txt not found')
